# Azure AI Agent Service with OpenAPI Specified Tools

## Verify that your TripAdvisor API works 

In [1]:
import requests, os

url = "https://api.content.tripadvisor.com/api/v1/location/search"
params = {
    "key": os.environ["TRIPADVISOR_API_KEY"],
    "searchQuery": "new york",
    "language": "en"
}
headers = {
    "accept": "application/json"
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    print(response.json())  # Print the JSON response
else:
    print(f"Error: {response.status_code}, {response.text}")

{'data': [{'location_id': '60763', 'name': 'New York City', 'address_obj': {'state': 'New York', 'country': 'United States', 'address_string': 'New York City, NY'}}, {'location_id': '91904', 'name': 'New York - New York Hotel & Casino', 'address_obj': {'street1': '3790 S Las Vegas Blvd', 'city': 'Las Vegas', 'state': 'Nevada', 'country': 'United States', 'postalcode': '89109-4338', 'address_string': '3790 S Las Vegas Blvd, Las Vegas, NV 89109-4338'}}, {'location_id': '112064', 'name': 'Warwick New York', 'address_obj': {'street1': '65 W 54th St', 'street2': '', 'city': 'New York City', 'state': 'New York', 'country': 'United States', 'postalcode': '10019', 'address_string': '65 W 54th St, New York City, NY 10019'}}, {'location_id': '28953', 'name': 'New York', 'address_obj': {'country': 'United States', 'address_string': 'NY'}}, {'location_id': '970327', 'name': 'New York Cafe', 'address_obj': {'street1': 'Erzsebet Korut 9-11', 'street2': 'Anantara New York Palace Budapest Hotel', 'cit

## Step 1: Create a project client

In [2]:
import os
import jsonref
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import OpenApiTool, OpenApiConnectionAuthDetails, OpenApiConnectionSecurityScheme


# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

## Step 2: Create the OpenAPI Spec tool definition

In [3]:
with open('./tripadvisor_openapi.json', 'r') as f:
    openapi_spec = jsonref.loads(f.read())

print(openapi_spec)

tripadvisor_connection = project_client.connections.get(
    connection_name="tripadvisor"
)
conn_id = tripadvisor_connection.id
print(conn_id)

auth = OpenApiConnectionAuthDetails(security_scheme=OpenApiConnectionSecurityScheme(connection_id=conn_id))

openapi = OpenApiTool(name="tripadvisor", spec=openapi_spec, description="get hotel and restaurant reviews of a location", auth=auth)

{'openapi': '3.0.1', 'servers': [{'url': 'https://api.content.tripadvisor.com/api'}], 'info': {'version': '1.0.0', 'title': 'Content API - TripAdvisor(Knowledge)', 'description': 'SSP includes Locations Details, Locations Photos, Locations Reviews, Location Search'}, 'paths': {'/v1/location/{locationId}/details': {'get': {'summary': 'Location Details', 'description': 'A Location Details request returns comprehensive information about a location (hotel, restaurant, or an attraction) such as name, address, rating, and URLs for the listing on Tripadvisor.', 'operationId': 'getLocationDetails', 'tags': ['Location Details'], 'parameters': [{'name': 'locationId', 'in': 'path', 'description': 'A unique identifier for a location on Tripadvisor. The location ID can be obtained using the Location Search.', 'required': True, 'schema': {'type': 'integer', 'format': 'int32'}}, {'name': 'language', 'in': 'query', 'description': 'The language in which to return results (e.g. "en" for English or "es" 

## Step 3: Create an agent and a thread

In [4]:
# Create agent with OpenAPI tool and process assistant run
agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="my-tripadvisor-assistant",
    instructions="You are a helpful assistant that will information using the tools provided to you.",
    tools=openapi.definitions
)
print(f"Created agent, ID: {agent.id}")

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

Created agent, ID: asst_5n8GDK6WwISUPpbVY2M14D3f
Created thread, ID: thread_Gfhrxn45ZMpHfU17XCiq29JJ


## Helper Function
- Add a message to the thread
- Run the Agent
- Check the Run Status
- Display the Agent's Response


In [5]:
def run_agent(user_input):  
    # Step 4: Add a message to the thread  
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=user_input,
    )
    print(f"Created message, ID: {message.id}")

    # Step 5 & 6: Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Step 7: Display the Agent's Response
    elif run.status == 'completed':
            # Fetch all messages in the thread
            messages = project_client.agents.list_messages(thread_id=thread.id)
            if messages.data:
                agent_message = messages.data[0]  # Get the last assistant message
                print(f"Agent Response: {agent_message.content[0].text.value}") 
            else:
                print("No messages found.")

## Running the agent using the Helper Function

In [8]:
user_input = "What is the La Grande Boucherie?"
run_agent(user_input)

Created message, ID: msg_Fd2ZHOPVKSkMIkcbwvGGWMb1
Agent Response: La Grande Boucherie refers to multiple locations. Here are the top ones:

1. **La Grande Boucherie (New York City, NY)**  
   Address: 145 W 53rd St, New York City, NY 10019-6005, United States

2. **La Grande Boucherie (Chicago, IL)**  
   Address: 431 N. Dearborn Street, Chicago, IL 60654, United States

3. **La Grande Boucherie (Washington DC)**  
   Address: 699 14th Street NW, Washington DC, DC 20005, United States

If you are thinking of a specific location, let me know to explore further details for you.


In [9]:
user_input = "What is the review of the La Grande Boucherie in New York City?"
run_agent(user_input)

Created message, ID: msg_UErcau1bqLoR8R4lfVGXlkde
Agent Response: Here are some recent reviews for **La Grande Boucherie** in New York City:

1. **Great hospitality and food!**  
   Rated: ★★★★★  
   *"Great date night option. Took my misses here for her birthday. The hospitality was top-tier, and we're looking forward to coming back."*  
   [Read more](https://www.tripadvisor.com/ShowUserReviews-g60763-d21410713-r1001448815-Reviews-La_Grande_Boucherie-New_York_City_New_York.html?m=66827#review1001448815)  

2. **Perfect night out**  
   Rated: ★★★★★  
   *"Celebrated my birthday here—fantastic food, warm atmosphere, and attentive staff. They even surprised me with a thoughtful birthday treat. Highly recommended for special occasions!"*  
   [Read more](https://www.tripadvisor.com/ShowUserReviews-g60763-d21410713-r1001425870-Reviews-La_Grande_Boucherie-New_York_City_New_York.html?m=66827#review1001425870)  

3. **Great birthday brunch**  
   Rated: ★★★★★  
   *"Had a large party of 10,

## Delete Agent to free up resources

In [9]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
